## Predicting Columns in a Table - Quick Start


Source: https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html 

Via a simple fit() call, AutoGluon can produce highly-accurate models to predict the values in one column of a data table based on the rest of the columns’ values. Use AutoGluon with tabular data for both classification and regression problems. This tutorial demonstrates how to use AutoGluon to produce a classification model that predicts whether or not a person’s income exceeds $50,000.

To start, import AutoGluon’s TabularPredictor and TabularDataset classes:

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

Load training data from a CSV file into an AutoGluon Dataset object. This object is essentially equivalent to a Pandas DataFrame and the same methods can be applied to both.

In [2]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

/Users/caihaocui/opt/miniconda3/envs/AutoGluon/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


Note that we loaded data from a CSV file stored in the cloud (AWS s3 bucket), but you can you specify a local file-path instead if you have already downloaded the CSV file to your own machine (e.g., using wget). Each row in the table train_data corresponds to a single training example. In this particular dataset, each row corresponds to an individual person, and the columns contain various characteristics reported during a census.

Let’s first use these features to predict whether the person’s income exceeds $50,000 or not, which is recorded in the class column of this table.

In [3]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object
/Users/caihaocui/opt/miniconda3/envs/AutoGluon/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Now use AutoGluon to train multiple models:



In [4]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    42574.18 MB
	Train

Next, load separate test data to demonstrate how to make predictions on new examples at inference time:

In [5]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


We use our trained models to make predictions on the new data and then evaluate performance:



In [6]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/Users/caihaocui/opt/miniconda3/envs/AutoGluon/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Evaluation: accuracy on test data: 0.8397993653393387
Evaluations on test data:
{
    "accuracy": 0.8397993653393387,
    "balanced_accuracy": 0.7437076677780596,
    "mcc": 0.5295565206264157,
    "f1": 0.6242496998799519,
    "precision": 0.7038440714672441,
    "recall": 0.5608283002588438
}
Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


We can also evaluate the performance of each individual trained model on our (labeled) test data:

In [8]:
predictor.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.842666,0.85,0.074881,0.006080,0.437865,0.074881,0.006080,0.437865,1,True,11
1,RandomForestGini,0.841335,0.84,0.076104,0.048978,0.468868,0.076104,0.048978,0.468868,1,True,5
2,RandomForestEntr,0.840721,0.83,0.074374,0.041690,0.407430,0.074374,0.041690,0.407430,1,True,6
3,LightGBM,0.839799,0.85,0.041230,0.012507,0.838760,0.041230,0.012507,0.838760,1,True,4
4,WeightedEnsemble_L2,0.839799,0.85,0.043618,0.013101,1.222406,0.002388,0.000594,0.383646,2,True,14
5,LightGBMXT,0.839390,0.83,0.021409,0.011018,0.992941,0.021409,0.011018,0.992941,1,True,3
6,CatBoost,0.837957,0.84,0.017557,0.009680,0.683572,0.017557,0.009680,0.683572,1,True,7
7,ExtraTreesEntr,0.834783,0.82,0.080310,0.040474,0.407589,0.080310,0.040474,0.407589,1,True,9
8,ExtraTreesGini,0.834476,0.82,0.101334,0.051537,0.410919,0.101334,0.051537,0.410919,1,True,8
9,LightGBMLarge,0.827823,0.83,0.033390,0.010490,1.293613,0.033390,0.010490,1.293613,1,True,13


Now you’re ready to try AutoGluon on your own tabular datasets! As long as they’re stored in a popular format like CSV, you should be able to achieve strong predictive performance with just 2 lines of code:

```python
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=<variable-name>).fit(train_data=<file-name>)
```

Description of fit():

Here we discuss what happened during fit().

Since there are only two possible values of the class variable, this was a binary classification problem, for which an appropriate performance metric is accuracy. AutoGluon automatically infers this as well as the type of each feature (i.e., which columns contain continuous numbers vs. discrete categories). AutogGluon can also automatically handle common issues like missing data and rescaling feature values.

We did not specify separate validation data and so AutoGluon automatically choses a random training/validation split of the data. The data used for validation is seperated from the training data and is used to determine the models and hyperparameter-values that produce the best results. Rather than just a single model, AutoGluon trains multiple models and ensembles them together to ensure superior predictive performance.

By default, AutoGluon tries to fit various types of models including neural networks and tree ensembles. Each type of model has various hyperparameters, which traditionally, the user would have to specify. AutoGluon automates this process.

AutoGluon automatically and iteratively tests values for hyperparameters to produce the best performance on the validation data. This involves repeatedly training models under different hyperparameter settings and evaluating their performance. This process can be computationally-intensive, so fit() can parallelize this process across multiple threads (and machines if distributed resources are available). To control runtimes, you can specify various arguments in fit() as demonstrated in the subsequent In-Depth tutorial.

For tabular problems, fit() returns a Predictor object. For classification, you can easily output predicted class probabilities instead of predicted classes:



In [10]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

,<=50K,>50K
0,0.949797,0.050203
1,0.945973,0.054027
2,0.433299,0.566701
3,0.991393,0.008607
4,0.949908,0.050092


Besides inference, this object can also summarize what happened during fit.


In [19]:
results = predictor.fit_summary()


*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2   0.900294       0.352193  10.805357                0.001113           1.266509            2       True         14
1     ExtraTreesEntr_BAG_L1   0.893912       0.092358   0.394143                0.092358           0.394143            1       True          9
2     ExtraTreesGini_BAG_L1   0.892927       0.092565   0.392809                0.092565           0.392809            1       True          8
3           CatBoost_BAG_L1   0.887489       0.043027   5.326382                0.043027           5.326382            1       True          7
4   RandomForestEntr_BAG_L1   0.886301       0.091141   0.389914                0.091141           0.389914            1       True          6
5   RandomForestGini_BAG_L1   0.884698       0.091445   0.402679                

From this summary, we can see that AutoGluon trained many different types of models as well as an ensemble of the best-performing models. The summary also describes the actual models that were trained during fit and how well each model performed on the held-out validation data. We can view what properties AutoGluon automatically inferred about our prediction task:

In [12]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', []) : 8 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])      : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
/Users/caihaocui/opt/miniconda3/envs/AutoGluon/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AutoGluon correctly recognized our prediction problem to be a binary classification task and decided that variables such as age should be represented as integers, whereas variables such as workclass should be represented as categorical objects. The feature_metadata attribute allows you to see the inferred data type of each predictive variable after preprocessing (this is it’s raw dtype; some features may also be associated with additional special dtypes if produced via feature-engineering, e.g. numerical representations of a datetime/text column).

We can evaluate the performance of each individual trained model on our (labeled) test data:

In [13]:
predictor.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.842666,0.85,0.077081,0.006080,0.437865,0.077081,0.006080,0.437865,1,True,11
1,RandomForestGini,0.841335,0.84,0.076173,0.048978,0.468868,0.076173,0.048978,0.468868,1,True,5
2,RandomForestEntr,0.840721,0.83,0.075019,0.041690,0.407430,0.075019,0.041690,0.407430,1,True,6
3,LightGBM,0.839799,0.85,0.027496,0.012507,0.838760,0.027496,0.012507,0.838760,1,True,4
4,WeightedEnsemble_L2,0.839799,0.85,0.029425,0.013101,1.222406,0.001929,0.000594,0.383646,2,True,14
5,LightGBMXT,0.839390,0.83,0.022646,0.011018,0.992941,0.022646,0.011018,0.992941,1,True,3
6,CatBoost,0.837957,0.84,0.017645,0.009680,0.683572,0.017645,0.009680,0.683572,1,True,7
7,ExtraTreesEntr,0.834783,0.82,0.079481,0.040474,0.407589,0.079481,0.040474,0.407589,1,True,9
8,ExtraTreesGini,0.834476,0.82,0.083868,0.051537,0.410919,0.083868,0.051537,0.410919,1,True,8
9,LightGBMLarge,0.827823,0.83,0.028883,0.010490,1.293613,0.028883,0.010490,1.293613,1,True,13


When we call predict(), AutoGluon automatically predicts with the model that displayed the best performance on validation data (i.e. the weighted-ensemble). We can instead specify which model to use for predictions like this:

In [14]:
predictor.predict(test_data, model='LightGBM')


/Users/caihaocui/opt/miniconda3/envs/AutoGluon/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object

Above the scores of predictive performance were based on a default evaluation metric (accuracy for binary classification). Performance in certain applications may be measured by different metrics than the ones AutoGluon optimizes for by default. If you know the metric that counts in your application, you should specify it as demonstrated in the next section.

## Maximizing predictive performance
Note: You should not call fit() with entirely default arguments if you are benchmarking AutoGluon-Tabular or hoping to maximize its accuracy! To get the best predictive accuracy with AutoGluon, you should generally use it like this:

In [15]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(test_data, silent=True)

No path specified. Models will be saved in: "AutogluonModels/ag-20210515_021854/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20210515_021854/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Us

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.902783,0.887489,0.070385,0.043027,5.326382,0.070385,0.043027,5.326382,1,True,7
1,LightGBMXT_BAG_L1,0.900161,0.881380,0.208763,0.054929,3.156310,0.208763,0.054929,3.156310,1,True,3
2,WeightedEnsemble_L2,0.894506,0.900294,0.991615,0.352193,10.805357,0.002748,0.001113,1.266509,2,True,14
3,LightGBM_BAG_L1,0.892347,0.866991,0.117996,0.051534,3.252953,0.117996,0.051534,3.252953,1,True,4
4,XGBoost_BAG_L1,0.891681,0.866575,0.363181,0.030624,1.932197,0.363181,0.030624,1.932197,1,True,11
5,RandomForestEntr_BAG_L1,0.888119,0.886301,0.088699,0.091141,0.389914,0.088699,0.091141,0.389914,1,True,6
6,RandomForestGini_BAG_L1,0.886598,0.884698,0.078156,0.091445,0.402679,0.078156,0.091445,0.402679,1,True,5
7,NeuralNetFastAI_BAG_L1,0.882037,0.866565,0.731166,0.123130,3.425514,0.731166,0.123130,3.425514,1,True,10
8,ExtraTreesGini_BAG_L1,0.881065,0.892927,0.094910,0.092565,0.392809,0.094910,0.092565,0.392809,1,True,8
9,ExtraTreesEntr_BAG_L1,0.880851,0.893912,0.092406,0.092358,0.394143,0.092406,0.092358,0.394143,1,True,9


This command implements the following strategy to maximize accuracy:

Specify the argument presets='best_quality', which allows AutoGluon to automatically construct powerful model ensembles based on stacking/bagging, and will greatly improve the resulting predictions if granted sufficient training time. The default value of presets is 'medium_quality_faster_train', which produces less accurate models but facilitates faster prototyping. With presets, you can flexibly prioritize predictive accuracy vs. training/inference speed. For example, if you care less about predictive performance and want to quickly deploy a basic model, consider using: presets=['good_quality_faster_inference_only_refit', 'optimize_for_deployment'].

- Provide the eval_metric if you know what metric will be used to evaluate predictions in your application. Some other non-default metrics you might use include things like: 'f1' (for binary classification), 'roc_auc' (for binary classification), 'log_loss' (for classification), 'mean_absolute_error' (for regression), 'median_absolute_error' (for regression). You can also define your own custom metric function, see examples in the folder: autogluon/core/metrics/

- Include all your data in train_data and do not provide tuning_data (AutoGluon will split the data more intelligently to fit its needs).

- Do not specify the hyperparameter_tune_kwargs argument (counterintuitively, hyperparameter tuning is not the best way to spend a limited training time budgets, as model ensembling is often superior). We recommend you only use hyperparameter_tune_kwargs if your goal is to deploy a single model rather than an ensemble.

- Do not specify hyperparameters argument (allow AutoGluon to adaptively select which models/hyperparameters to use).

- Set time_limit to the longest amount of time (in seconds) that you are willing to wait. AutoGluon’s predictive performance improves the longer fit() is allowed to run.

## Regression (predicting numeric table columns):
To demonstrate that fit() can also automatically handle regression tasks, we now try to predict the numeric age variable in the same table based on the other features:

In [16]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

Summary of age variable: 
 count    500.00000
mean      39.65200
std       13.52393
min       17.00000
25%       29.00000
50%       38.00000
75%       49.00000
max       85.00000
Name: age, dtype: float64
/Users/caihaocui/opt/miniconda3/envs/AutoGluon/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We again call fit(), imposing a time-limit this time (in seconds), and also demonstrate a shorthand method to evaluate the resulting model on the test data (which contain labels):

In [17]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (85, 17, 39.652, 13.52393)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    42354.25 MB
	Train Data (Original)  Memory Usage: 0.32 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...

Note that we didn’t need to tell AutoGluon this is a regression problem, it automatically inferred this from the data and reported the appropriate performance metric (RMSE by default). To specify a particular evaluation metric other than the default, set the eval_metric argument of fit() and AutoGluon will tailor its models to optimize your metric (e.g. eval_metric = 'mean_absolute_error'). For evaluation metrics where higher values are worse (like RMSE), AutoGluon may sometimes flips their sign and print them as negative values during training (as it internally assumes higher values are better).

We can call leaderboard to see the per-model performance:

In [18]:
predictor_age.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-10.534319,-11.313772,1.273746,0.230337,7.705367,0.004589,0.000488,0.409813,2,True,12
1,ExtraTreesMSE,-10.691115,-11.480752,0.072538,0.035789,0.365046,0.072538,0.035789,0.365046,1,True,7
2,RandomForestMSE,-10.746518,-11.602848,0.068356,0.034627,0.373121,0.068356,0.034627,0.373121,1,True,5
3,LightGBMXT,-10.753344,-11.814712,0.045171,0.012330,1.172688,0.045171,0.012330,1.172688,1,True,3
4,CatBoost,-10.800412,-11.744795,0.019409,0.009434,0.678361,0.019409,0.009434,0.678361,1,True,6
5,LightGBM,-10.972156,-11.929546,0.023109,0.012057,0.840785,0.023109,0.012057,0.840785,1,True,4
6,XGBoost,-11.121008,-12.174270,0.091314,0.006893,0.582656,0.091314,0.006893,0.582656,1,True,9
7,LightGBMLarge,-11.598649,-12.167606,0.050763,0.011821,2.860899,0.050763,0.011821,2.860899,1,True,11
8,NeuralNetMXNet,-13.003221,-13.169344,0.949260,0.118719,3.282897,0.949260,0.118719,3.282897,1,True,10
9,KNeighborsUnif,-14.902058,-15.686937,0.014174,0.012266,0.002818,0.014174,0.012266,0.002818,1,True,1


Data Formats: AutoGluon can currently operate on data tables already loaded into Python as pandas DataFrames, or those stored in files of CSV format or Parquet format. If your data live in multiple tables, you will first need to join them into a single table whose rows correspond to statistically independent observations (datapoints) and columns correspond to different features (aka. variables/covariates).

Refer to the TabularPredictor documentation to see all of the available methods/options.